In [1]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from transformers import AutoModel, AutoTokenizer

In [2]:
import sys
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(os.path.abspath(src_dir))

from importlib import reload

import src.utils.data_loader

reload(src.utils.data_loader)

from src.utils.data_loader import DataEncoderNER

In [3]:
train_df = pd.read_csv('../data/dataset/ner_train_dep_and_regions.csv', sep=',')
val_df = pd.read_csv('../data/dataset/ner_valid_dep_and_regions.csv', sep=',')
# test_df = pd.read_csv('../data/dataset/ner_test.csv', sep=',')

In [4]:
batch_size = 30
epochs = 1
learning_rate = 2e-5

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-french-europeana-cased")
bert_model = AutoModel.from_pretrained("dbmdz/bert-base-french-europeana-cased")
max_len = 128

In [5]:
le_sub_sentences = LabelEncoder()

classes = ["I", "O", "B"]
le_sub_sentences.fit(classes)

train_df['status'] = le_sub_sentences.transform(train_df['status'])
val_df['status'] = le_sub_sentences.transform(val_df['status'])
# test_df['status'] = le_sub_sentences.transform(test_df['status'])

In [6]:
train_dataset = DataEncoderNER(train_df["sub_sentence"], train_df["status"], tokenizer, max_len)
val_dataset = DataEncoderNER(val_df["sub_sentence"], val_df["status"], tokenizer, max_len)

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [7]:
class BertForSequenceClassification(nn.Module):
    def __init__(self, n_classes):
        super(BertForSequenceClassification, self).__init__()
        self.bert = bert_model
        self.drop = nn.Dropout(p=0.3)
        self.out_linear = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs[1]

        output_depart = self.out_linear(self.drop(pooled_output))

        return output_depart

In [8]:
model = BertForSequenceClassification(len(le_sub_sentences.classes_))
model.load_state_dict(torch.load("./processed/city_nlp/NER_model.pth"))
model = model.to(device)
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=learning_rate
)
loss_fn = nn.CrossEntropyLoss().to(device)

C:\Users\Danyl\AppData\Local\Temp\ipykernel_12680\485327664.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./processed/city_nlp/NER_mo

In [9]:
def train_epoch(model, data_loader_train, data_loader_valid, loss_fn, optimizer, device, current_epoch):
    model = model.train()
    losses_train = []
    losses_valid = []
    correct_predictions_train = 0
    correct_predictions_valid = 0

    with tqdm(total=len(data_loader_train), desc=f"Epoch {current_epoch}", unit="batch") as pbar:
        for d in data_loader_train:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["status"].to(device)

            optimizer.zero_grad()

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            loss = loss_fn(outputs, labels)

            correct_predictions_train += (outputs.argmax(1) == labels).sum().item()
            losses_train.append(loss.item())

            loss.backward()
            optimizer.step()

            pbar.update(1)

    model = model.eval()

    for d in data_loader_valid:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels= d["status"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = loss_fn(outputs, labels)

        correct_predictions_valid += (outputs.argmax(1) == labels).sum().item()
        losses_valid.append(loss.item())

    train_acc = correct_predictions_train / len(data_loader_train.dataset)
    train_loss = np.mean(losses_train)

    valid_acc = correct_predictions_valid / len(data_loader_valid.dataset)
    valid_loss = np.mean(losses_valid)

    return {"train_acc": train_acc, "train_loss": train_loss, "valid_acc": valid_acc, "valid_loss": valid_loss}

In [10]:
history = []

for epoch in range(epochs):
    results = train_epoch(
        model,
        train_loader,
        val_loader,
        loss_fn,
        optimizer,
        device,
        epoch + 1
    )
    print(results)
    history.append(results)

Epoch 1:   0%|          | 0/4687 [00:00<?, ?batch/s]

{'train_acc': 0.9988620280083357, 'train_loss': 0.003646076771722706, 'valid_acc': 0.997984927255874, 'valid_loss': 0.007593254079664614}


In [11]:
history_df = pd.DataFrame(history)
# history_df[["train_acc", "valid_acc"]] = history_df[["train_acc", "valid_acc"]] * 2
file_name = "NER_model_train_dep_reg_fine_tuning"
history_df.to_csv(f"./processed/city_nlp/{file_name}.csv")
torch.save(model.state_dict(), f"./processed/city_nlp/{file_name}.pth")